# 单目标线性规划

决策变量，目标函数，约束条件
一直求最小值，如果要求最大值则乘符号变成最小值

- min 4_x1 + 3_x2 + 2_x3
- x1 + 2x2 + 3x3 >= 6
- 2x1 + x2 + x3 <= 8
- x1 >= 0, x2 >= 0, x3 >= 0

In [2]:
# 一直求最小值，如果要求最大值则乘符号变成最小值
from scipy.optimize import linprog

# 不等式约束的系数矩阵
epsilon = 1e-9  # 一个非常小的正数

# 目标函数的系数
c = [4, 3, 2]

# 不等式约束的系数矩阵
A_ub = [
    [-1, -2, -3],  # 对应于 x1 + 2x2 + 3x3 >= 6 转化为 -x1 - 2x2 - 3x3 <= -6
    [2, 1, 1]  # 对应于 2x1 + x2 + x3 <= 8
]
# 不等式约束的右边
b_ub = [-6, 8]

# 等式约束的系数矩阵
A_eq = [

]
# 等式约束的右边
b_eq = []

# 边界条件
bounds = [(0, None), (0, None), (0, None)]  # x1 >= 0, x2 >= 0, x3 >= 0

# 求解线性规划问题
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=None, b_eq=None, bounds=bounds, method='highs')

# 打印结果
if result.success:
    print(f"Optimal value: {result.fun}")
    print(f"Optimal solution: {result.x}")
else:
    print("No solution found.")

Optimal value: 4.0
Optimal solution: [0. 0. 2.]


In [1]:
import cvxpy as cp

x = cp.Variable((3,))

obj = cp.Minimize(4 * x[0] + 3 * x[1] + 2 * x[2])

con = [
    x[0] + 2 * x[1] + 3 * x[2] >= 6,
    2 * x[0] + x[1] + x[2] <= 8,
    x[0] >= 0,
    x[1] >= 0,
    x[2] >= 0,
]

problem = cp.Problem(obj, con)
problem.solve(solver=cp.CPLEX)
print('最优解为：', x.value)
print('最优值为：', problem.value)

最优解为： [0. 0. 2.]
最优值为： 4.0


# 多目标线性规划

多目标线性规划一般可以使用对多个目标函数加权、绘画多维平面（多少个目标函数就多少维）判断，这些都是要主观的设置参数或选择

绘画多维平面一般是对某个函数进行限制

In [ ]:
import numpy as np
from scipy.optimize import linprog

# 目标函数系数
c1 = [-1, -2]  # 目标函数1的系数（希望最大化）
c2 = [3, 2]  # 目标函数2的系数（希望最大化）

A_ub = [[1, 2], [2, 1]]
b_ub = [6, 6]
bounds = [(0, None), (0, None)]
# 优化目标函数1
result1 = linprog(c1, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print("Optimal value for objective 1:", -result1.fun)
print("Optimal solution for objective 1:", result1.x)

# 优化目标函数2
result2 = linprog(c2, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
print("Optimal value for objective 2:", result2.fun)
print("Optimal solution for objective 2:", result2.x)


# 中间解的插值
def interpolate(x1, x2, alpha):
    return [alpha * x1[i] + (1 - alpha) * x2[i] for i in range(len(x1))]


alpha_values = np.linspace(0, 1, num=5)
for alpha in alpha_values:
    x_interpolated = interpolate(result1.x, result2.x, alpha)
    # 计算中间解的目标函数值
    f1_value = -np.dot(c1, x_interpolated)
    f2_value = np.dot(c2, x_interpolated)
    print(f"Interpolated solution with alpha={alpha}: {x_interpolated}, f1={f1_value}, f2={f2_value}")

# 选址问题

In [2]:
from scipy.optimize import linprog

A = [
    [5, 1],
    [2, 7],
]
P = [
    [1.25, 1.25],
    [8.75, 0.75],
    [0.50, 4.75],
    [5.75, 5.00],
    [3.00, 6.50],
    [7.25, 7.25],
]
V = [3, 5, 4, 7, 6, 11]


def distance(x, y):
    return ((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2) ** 0.5


# 两个临时矿场到工地的距离，也是最终结果的系数
D = [distance(x, y) for x in A for y in P]

# 料场小于20
A_ub = [
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
]
b_ub = [20, 20]
# 日需求量
A_eq = [
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
]
b_eq = [3, 5, 4, 7, 6, 11]

bound = [
    (0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
    (0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
]

result = linprog(D, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bound, method='highs')
print(result.fun)
print(result.x)

135.28154179067644
[ 3.  5.  0.  7.  0.  1.  0.  0.  4.  0.  6. 10.]
